In [1]:
import pandas as pd
import pymongo
from nba_api.stats.static import teams

In [2]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.sportsDB

In [10]:
nfl_df = pd.read_csv("matchups_data.csv")
nfl_df = nfl_df.rename(columns={"TEAM NAME": "TEAM_NAME"})
nfl_df.head()

,TEAM_NAME,MATCHUP,LOCATION,WL,OPPONENT
0,Atlanta Falcons,Atlanta Falcons @ Arizona Cardinals,Away,L,Arizona Cardinals
1,Atlanta Falcons,Atlanta Falcons vs. Arizona Cardinals,Home,W,Arizona Cardinals
2,Baltimore Ravens,Baltimore Ravens @ Arizona Cardinals,Away,L,Arizona Cardinals
3,Baltimore Ravens,Baltimore Ravens vs. Arizona Cardinals,Home,W,Arizona Cardinals
4,Buffalo Bills,Buffalo Bills @ Arizona Cardinals,Away,W,Arizona Cardinals


In [18]:
grouped_df = nfl_df.groupby(['TEAM_NAME', 'OPPONENT','LOCATION', 'WL'], as_index=False)
winloss = grouped_df.count()
winloss.head(40)

,TEAM_NAME,OPPONENT,LOCATION,WL,MATCHUP
0,Arizona Cardinals,Atlanta Falcons,Away,L,3
1,Arizona Cardinals,Atlanta Falcons,Home,W,3
2,Arizona Cardinals,Buffalo Bills,Away,L,1
3,Arizona Cardinals,Buffalo Bills,Home,W,1
4,Arizona Cardinals,Carolina Panthers,Away,L,2
5,Arizona Cardinals,Carolina Panthers,Home,W,2
6,Arizona Cardinals,Chicago Bears,Away,W,1
7,Arizona Cardinals,Chicago Bears,Home,L,1
8,Arizona Cardinals,Cleveland Browns,Away,W,1
9,Arizona Cardinals,Cleveland Browns,Home,L,1


In [7]:
team_list = nfl_df['TEAM NAME'].unique().tolist()

In [16]:
teamname = "Arizona Cardinals"
opponent = "Atlanta Falcons"
superdict = {}
regdict = {}
littledict = {}
wins = 0
losses = 0
for index, row in winloss.iterrows():
    
    if row['TEAM_NAME'] == teamname:
        if row['OPPONENT'] == opponent:
            if row['LOCATION'] == 'Away':
                if row['WL'] == "W":
                    wins = row['MATCHUP']
                if row['WL'] == 'L':
                    losses = row['MATCHUP']
        if row['OPPONENT'] != opponent:
            littledict['wins'] = wins
            littledict['losses'] = losses
            regdict[opponent] = littledict
            opponent = row['OPPONENT']
            wins = 0
            losses = 0
            littledict = {}
            if row['LOCATION'] == 'Away':
                if row['WL'] == "W":
                    wins = row['MATCHUP']
                if row['WL'] == 'L':
                    losses = row['MATCHUP']
    elif row['TEAM_NAME'] != teamname:
        littledict['wins'] = wins
        littledict['losses'] = losses
        regdict[opponent] = littledict
        superdict[teamname] = regdict
        regdict = {}
        littledict = {}
        wins = 0
        losses = 0
        teamname = row['TEAM_NAME']
        opponent = row['OPPONENT']
        
        if row['WL'] == "W":
            wins = row['MATCHUP']
        if row['WL'] == 'L':
            losses = row['MATCHUP']
littledict['wins'] = wins
littledict['losses'] = losses
regdict[opponent] = littledict
superdict[teamname] = regdict

In [26]:
del superdict['Arizona Cardinals']
superdict

{' Arizona Cardinals': {' Atlanta Falcons': {'wins': 0, 'losses': 3},
  ' Buffalo Bills': {'wins': 0, 'losses': 1},
  ' Carolina Panthers': {'wins': 0, 'losses': 2},
  ' Chicago Bears': {'wins': 1, 'losses': 0},
  ' Cleveland Browns': {'wins': 1, 'losses': 0},
  ' Dallas Cowboys': {'wins': 1, 'losses': 0},
  ' Denver Broncos': {'wins': 0, 'losses': 1},
  ' Detroit Lions': {'wins': 1, 'losses': 0},
  ' Green Bay Packers': {'wins': 0, 'losses': 1},
  ' Jacksonville Jaguars': {'wins': 1, 'losses': 0},
  ' Miami Dolphins': {'wins': 0, 'losses': 1},
  ' Minnesota Vikings': {'wins': 0, 'losses': 2},
  ' New England Patriots': {'wins': 1, 'losses': 0},
  ' New Orleans Saints': {'wins': 0, 'losses': 1},
  ' New York Giants': {'wins': 1, 'losses': 0},
  ' New York Jets': {'wins': 0, 'losses': 1},
  ' Oakland Raiders': {'wins': 1, 'losses': 0},
  ' Philadelphia Eagles': {'wins': 1, 'losses': 1},
  ' Pittsburgh Steelers': {'wins': 0, 'losses': 1},
  ' San Francisco 49ers': {'wins': 2, 'losses': 3

In [27]:
superdict[' Arizona Cardinals']

{' Atlanta Falcons': {'wins': 0, 'losses': 3},
 ' Buffalo Bills': {'wins': 0, 'losses': 1},
 ' Carolina Panthers': {'wins': 0, 'losses': 2},
 ' Chicago Bears': {'wins': 1, 'losses': 0},
 ' Cleveland Browns': {'wins': 1, 'losses': 0},
 ' Dallas Cowboys': {'wins': 1, 'losses': 0},
 ' Denver Broncos': {'wins': 0, 'losses': 1},
 ' Detroit Lions': {'wins': 1, 'losses': 0},
 ' Green Bay Packers': {'wins': 0, 'losses': 1},
 ' Jacksonville Jaguars': {'wins': 1, 'losses': 0},
 ' Miami Dolphins': {'wins': 0, 'losses': 1},
 ' Minnesota Vikings': {'wins': 0, 'losses': 2},
 ' New England Patriots': {'wins': 1, 'losses': 0},
 ' New Orleans Saints': {'wins': 0, 'losses': 1},
 ' New York Giants': {'wins': 1, 'losses': 0},
 ' New York Jets': {'wins': 0, 'losses': 1},
 ' Oakland Raiders': {'wins': 1, 'losses': 0},
 ' Philadelphia Eagles': {'wins': 1, 'losses': 1},
 ' Pittsburgh Steelers': {'wins': 0, 'losses': 1},
 ' San Francisco 49ers': {'wins': 2, 'losses': 3},
 ' Seattle Seahawks': {'wins': 3, 'loss